In [128]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In this link we can find several architectures for generative models
https://github.com/hwalsuklee/tensorflow-generative-model-collections

## Ideas to implement
> Modify the trainning architectre:
        for generator: train_g for disc: train disc

> Best practice: change the privacy of the variables -> ***Somewhat - Check***

> Moduralize the trainning of G and D in the loop  -> ***Check***
  
> <font color='red'> Resnet to GANS? => ResGans</font> Squeeze-and-Excitation Networks (SENets)??

> Can we apply penalty in both sides for faster convergence?

> Introduce the nosie generator inside the networks -> ***Check*** -> El generador mantiene el output size del input

> Define a late activation penalty function -> ***Check***

> Plot number of cases that are valid  -> ***Check***

> Modificar la arquitectura para que el penalty esté en el input

> Modificar las arquitecturas de entrenamiento de G y D

> Anadir DropOut a las NN. <font color='blue'> Tiene sentido implementar esto??</font>

> Probar diferentes tipos de activation function as PRelu and Softplus

> Add weight inizialization

> Tiene sentido variar el gp_param?

> Probar con 5 dimensiones una para cada penalty

> Provar a activar el penalty mas tarde

## Questions

> En que momento (dimensiones D)  aplicar el penalty input

> Funcionamiento del upsample -> Adapta la imagen input al size indicado interpolando. Se pueden usar varios métodos

> Tiene sentido implementar el Drop Out como método de regularización

> Por que diverege lossD y lossG cuando ha pasado un gran numero de rondas -> Da la sensación de que podría ser debido a que anteriormente he usado torch.cat con una doble entrada de input. El back propagation aquí no queda claro lo que hace. 

> Gran diferencia entre Batch Norm y Instance norm. ¿¿Por??

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from torch.utils import data
import torchvision
import sys
from copy import deepcopy
#from visdom import Visdom

from dataset import DeepmoonDataset

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
Tensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor


# Visualization
import time
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

#More libraries
import random
import numpy as np
import pandas as pd


#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print("Using GPU:" + str(use_cuda))

Using GPU:False


Fix random seed: Pretty sure that this still doesn't work

In [2]:
seed = 128

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed_all(seed)


Parameters

In [3]:
batch_size = 128
latent_size = 64     # Size of latent vector (i.e. size of generator input)
n_features = 32      # Size of feature maps in generator
lr = 0.0002
betas = (0.5, 0.999) # Why is this not 0.9,0.9999 as standard?
n_epochs = 500
n_critic = 5
n_print = 137
gamma = 0.8


Parameter for the WGAN

In [4]:
gp_param=1

Paramaters for the constraints

In [5]:
start_max = 2
end_max = 2
start_min = 1
end_min = 1
lmbda = 10  #Penalty parameter for constraints


In [6]:
def progressive_lambda(aIntEpoca,mode='constant'):
    ''' Method that depending on the mode it calculates a lambda coeficient, the variable modes depend in the epoch'''
    if mode == 'exponential':
        r_0=0.001
        tau=5
        lam=r_0*np.exp(1+aIntEpoca/tau)
        return lam
    if mode == 'exp_base1.2':
        r_0=0.005
        tau=5
        lam=r_0*1.2**(1+aIntEpoca/tau)
        return lam
    elif mode =='constant':
        return 10
    elif mode =='null':
        return 0
       
    else:
        return 0
progressive_lambda(100,mode='null')

0

Load data and split into training and validation sets

In [7]:
json_path = '../data/2016.json'
validation_split = 0.2

dataset = DeepmoonDataset(json_path)

split_len = [round((1-validation_split) * len(dataset)), round(validation_split * len(dataset))]
split = {x: y for x, y in zip(['train', 'val'], data.random_split(dataset, lengths=split_len))}

loader = {x: data.DataLoader(split[x], batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}


### Create the nn

Let's define a convolutional and a deep convolutional units

#### Generator

In [8]:
# Generator Code
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of feature maps in discriminator
ndf = 32

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( latent_size, n_features * 8, 4, 1, 0),
            nn.BatchNorm2d(n_features * 8),
            nn.ReLU(True),
            # state size. (n*8) x 4 x 4
            nn.ConvTranspose2d(n_features * 8, n_features * 4, 4, 2, 1),
            nn.BatchNorm2d(n_features * 4),
            nn.ReLU(True),
            # state size. (n*4) x 8 x 8
            nn.ConvTranspose2d( n_features * 4, n_features * 2, 4, 2, 1),
            nn.BatchNorm2d(n_features * 2),
            nn.ReLU(True),
            # state size. (n*2) x 16 x 16
            nn.ConvTranspose2d( n_features * 2, n_features, 4, 2, 1),
            nn.BatchNorm2d(n_features),
            nn.ReLU(True),
            # state size. (n) x 32 x 32
            nn.ConvTranspose2d( n_features, nc, 4, 2, 1, bias=False),
            nn.Sigmoid(),
            nn.Upsample(size=(18, 11)) #Esto reordena los datos
            
            
            # state size. (nc) x 64 x 64
        )

    def forward(self,asize):
        x = torch.randn(asize, latent_size, 1, 1, device=device)
        
        return self.main(x)
netG=Generator()
netG.to(device)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(64, 256, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Sigmoid()
    (14): Upsample(size=(18, 11), mode=nearest)
  )
)

In [9]:
netG(44).size()

torch.Size([44, 3, 18, 11])

#### Discriminator Network

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Upsample(size=(32, 32)),
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ndf * 2),
            #Hay una gran diferencia entre InstanceNormd y BatchNorm
            # https://stackoverflow.com/questions/45463778/instance-normalisation-vs-batch-normalisation
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            )
  
    def forward(self,x):        
        return self.main(x).view(x.size(0), -1)
    
netD = Discriminator()
netD.to(device)

Discriminator(
  (main): Sequential(
    (0): Upsample(size=(32, 32), mode=nearest)
    (1): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (2): LeakyReLU(negative_slope=0.2, inplace)
    (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): LeakyReLU(negative_slope=0.2, inplace)
    (6): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (8): LeakyReLU(negative_slope=0.2, inplace)
    (9): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

#### Optimizadores de las NN

In [11]:
optimizerD = torch.optim.Adam(netD.parameters(), lr, betas)
optimizerG = torch.optim.Adam(netG.parameters(), lr, betas)

Im not sure of what this function does

In [12]:
class RoundNoGradient(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        return x.round()
    @staticmethod
    def backward(ctx, g):
        return g


Define penalties

In [15]:
aa.size()

torch.Size([128, 3, 18, 11])

In [17]:
netG(44).size()

torch.Size([44, 3, 18, 11])

In [38]:
RoundNoGradient.apply(netG(1))

tensor([[[[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1.],
          [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
          [0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1.],
          [0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
          [0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.],
          [0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1.],
          [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1.],
          [1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1.],
          [0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
          [0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0.],
          [0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
          [0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.],
          [0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
          [1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0.],
          [0., 1., 1., 1., 1., 0., 1., 1., 0., 1

In [55]:
b=torch.sum(RoundNoGradient.apply(netG(128)),(2,3))[:,0]
print(b.size())

torch.relu(b)


torch.Size([128])


tensor([ 86.,  93.,  84.,  98.,  94.,  87.,  88.,  88.,  95.,  93.,  88.,  86.,
         83.,  90.,  87.,  98.,  81., 102.,  87.,  90.,  84.,  86.,  90.,  97.,
         90.,  91.,  90.,  90.,  85.,  91., 103.,  91.,  93.,  88.,  81.,  98.,
         87.,  90.,  89.,  91., 104.,  90.,  91.,  87.,  86.,  95.,  80.,  88.,
        101., 100.,  93.,  98.,  85.,  92.,  80.,  91., 104., 102.,  89.,  98.,
         98.,  95.,  89.,  98.,  85.,  82., 102., 106.,  85.,  87.,  88.,  95.,
         85.,  99.,  89.,  96.,  94.,  94.,  83.,  84.,  92.,  88.,  87.,  99.,
         89.,  90.,  90.,  87., 101.,  91.,  89.,  84.,  95.,  85.,  91.,  77.,
         93.,  82.,  90.,  93.,  90., 103.,  87.,  87.,  90.,  96.,  77.,  94.,
         74.,  90.,  88.,  86.,  93.,  97.,  94.,  88.,  87.,  89.,  93.,  87.,
         95.,  88.,  91.,  93.,  94.,  89., 100.,  97.],
       grad_fn=<ReluBackward0>)

In [40]:
print(aa.size())
torch.mean(aa)

torch.Size([128, 3, 18, 11])


tensor(0.0133)

In [41]:
torch.mean(netG(128))

tensor(0.5156, grad_fn=<MeanBackward1>)

In [42]:
def channel_penalty(x_fake):
    # Este penalty recibe el input de la fake y comprueba si contiene el los iniciales y los finales adecuados
    # devuelve un array del tamaño del input inicial
    
    count = torch.sum(RoundNoGradient.apply(x_fake), (2, 3))

    count_start = count[:, 0]
    count_end = count[:, 2]

    g_start_max = torch.relu(count_start - start_max) ** 2
    g_end_max = torch.relu(count_end - end_max) ** 2

    g_start_min = torch.relu(-count_start + start_min) ** 2
    g_end_min = torch.relu(-count_end + end_min) ** 2
    return (g_start_max + g_end_max + g_start_min + g_end_min).view(x_fake.size(0), -1)

def duplicate_penalty(x_fake):
    count = torch.sum(RoundNoGradient.apply(x_fake), 1, keepdim=True)
    g = torch.sum(torch.relu(count - 1) ** 2, (2, 3))
    return g.view(x_fake.size(0), -1)

def gradient_penalty(discriminator_net, x_real, x_fake):
    batch_size = x_real.size(0)
    # error*real +(1-error)*false -> Se suman ambas con un factor ruido
    epsilon = torch.rand(batch_size, 1, 1, 1, device=device)
    x_merged = (epsilon * x_real + (1 - epsilon) * x_fake).requires_grad_(True)
    
    #Se evalua el resultado de la suma de la combinacion linbeal de ambas
    out_merged = discriminator_net(x_merged)

    grad, = torch.autograd.grad(
        outputs=out_merged,
        inputs=x_merged,
        grad_outputs=Tensor(batch_size, 1).fill_(1),
        create_graph=True,
        retain_graph=True,
        only_inputs=True)

    return ((grad.view(batch_size, -1).norm(2, dim=1) - 1)**2).mean()

def avg_start_end(x_fake):
    count = torch.sum(x_fake.round(), (2, 3))

    count_start = count[:, 0]
    count_end = count[:, 2]

    count_start.apply_(lambda x: 1 if start_max >= x >= start_min else 0)
    count_end.apply_(lambda x: 1 if end_max >= x >= end_min else 0)

    valid_solutions = Tensor([1 if s == 1 and e == 1 else 0
                              for s, e in zip(count_start, count_end)])

    return count_start.mean(), count_end.mean(), valid_solutions.mean()


In [53]:
for i, data in enumerate(loader['train']):

        x_real = data['moves'].type(Tensor)
        if channel_penalty(x_real).sum()>0:
            print(i)
            print('channel')
            print(channel_penalty(x_real).sum())
        if duplicate_penalty(x_real).sum()>0:
            print('duplicate')
            print(duplicate_penalty(x_real).sum())


44
channel
tensor(1.)


In [142]:
def penalty_extra_channel(image):
    '''This method receives an image and returns it with one channel more.
       This new channel has all the values the same and equal to the penalty'''
    
    # We add the penalty
    pen_c = lmbda * channel_penalty(image)
    pen_c = pen_c.unsqueeze(-1).unsqueeze(-1).expand(image.size(0), 1,image.size(2), image.size(3))
    
    
    
    out=torch.cat((image,pen_c), 1)
    
    pen_d = lmbda * duplicate_penalty(image)
    
    
    pen_d = pen_d.unsqueeze(-1).unsqueeze(-1).expand(image.size(0), 1,image.size(2), image.size(3))
    
    return torch.cat((out,pen_d), 1)
    

In [143]:
netG(128).size()

torch.Size([128, 5, 18, 11])

In [144]:
penalty_extra_channel(netG(128)).size()

torch.Size([128, 7, 18, 11])

In [145]:
def valid_solution_percent(imagen_falsa):
    ''' From an image it calculates the % of images that satisfy the penalty'''
    tens=channel_penalty(imagen_falsa) + duplicate_penalty(imagen_falsa)
    
    return (sum(1 for i in tens if i == 0 )/len(tens))*10 #We multiply this by 10 to scale it more visuable
    

#### Trainning of the Generator and Discriminator

In [61]:
print(aa.size())
torch.mean(aa)

torch.Size([128, 3, 18, 11])


tensor(0.0133)

In [13]:
def train_generator(optim,real_data,epoch):
    netG.zero_grad()

    # Creamos una imagen falsa
    x_fake = netG(real_data.size(0))
    
    # Calculamos el error del discriminador anadiento tanto la imagen falsa como el penalty
    out_fake = netD(x_fake)
    
   
    valid_solutions=valid_solution_percent(x_fake)                       
    

    lossG = -torch.mean(out_fake) \

    lossG.backward()
    optim.step()
    
    return lossG,valid_solutions

for i, data in enumerate(loader['train']):


    aa = data['moves'].type(Tensor)
    print(aa.size())
    aa = penalty_extra_channel(aa)
    a,b=train_generator(optimizerG,aa,113)
    
    break
    

torch.Size([128, 3, 18, 11])


NameError: name 'penalty_extra_channel' is not defined

In [147]:
aa.size()

torch.Size([128, 5, 18, 11])

In [148]:
pen_c=lmbda * (channel_penalty(aa))
print(pen_c.size())
pen_c = pen_c.unsqueeze(-1).unsqueeze(-1).expand(aa.size(0), 1,aa.size(2), aa.size(3))
pen_c.size()

torch.Size([128, 1])


torch.Size([128, 1, 18, 11])

In [149]:
penalty_extra_channel(aa)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 1., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 

In [150]:
for name, param in netD.named_parameters():
    if param.requires_grad:
        if name=='fc.0.bias' or name=='fc.0.weight':
            print (name,param)

In [151]:
def train_discriminator(optim,real_data):
   
    netD.zero_grad()
    
    x_fake = netG(real_data.size(0))
    
    #Hacemos la predicción de la imagen con el discriminador para el real y el fake
    out_real = netD(real_data)
    out_fake = netD(x_fake)
    

    #Anadimos la loss function interpolando entre la media del error de la real y de fake
    # introducimos el valor del penalty
    error_d = torch.mean(out_fake) - torch.mean(out_real) + gp_param * gradient_penalty(netD, real_data.data, x_fake.data) 
    #Hacemos el backward propagation y damos un step en la dirección del gradiente 
        # del discriminador
    error_d.backward(retain_graph=True)
    optim.step()
   
    return error_d

for i, data in enumerate(loader['train']):
    epoch=30


    aa = data['moves'].type(Tensor)
    aa=penalty_extra_channel(aa)
    
    a=train_discriminator(optimizerD,aa)
    print(a)
    break



    

tensor(22.8962, grad_fn=<AddBackward0>)


Start trainning

In [152]:
# Cummulative Lists
fixed_noise = torch.randn(128, latent_size, 1, 1, device=device)

history = {'lossD': [], 'lossG': [], 'avgStart': [], 'avgEnd': [],
           'valid': [], 'validHist': [], 'imgs': [],'epoch': [],
           'porcentaje' : []}

epoch=-1 #Instanciamos esta variable fuera para poder repetir entrenamiento

In [153]:
while epoch<n_epochs:
    epoch+=1

    for i, data in enumerate(loader['train']):

        x_real = data['moves'].type(Tensor)
        x_real = penalty_extra_channel(x_real)
        N = x_real.size(0)
        
        lossD = train_discriminator(optimizerD,x_real)


        # En un número ciclico de veces entrenamos el Generador
        if i % n_critic == 0:

            lossG,perc = train_generator(optimizerG,x_real,epoch)

        # Print training stats
        if i % n_print == 0 and epoch>1:
            
            plt.figure()
            
            #plt.plot(history['lossD'],'r',label='loss_Discriminator')
            #plt.plot(history['lossG'],'k',label='loss_Generator')
            plt.plot(history['porcentaje'],label='Percentage')
            
            plt.legend(loc=(0,0.8))
           

            
            plt.title('Porcentaje in epoch {} with 4 channels'.format(epoch))
            plt.savefig('../imagenes/with4channels'.format(epoch))
            
            display.clear_output(wait=False)
            display.display(plt.gcf())
            time.sleep(1)
            print(
                "[Epoch {:5}/{:5}] [Batch {:3}/{:3}] [D loss: {:2.6f}] [G loss: {:2.6f}] percent {}".format(
                    epoch, n_epochs, i, len(loader['train']), lossD.item(), lossG.item(),perc/10
                )
            )


    # Save losses in history and update plot
    with torch.no_grad():
        history['imgs'] += [netG(x_real.size(0)).detach().cpu()]
        avgStart, avgEnd, valid = avg_start_end(history['imgs'][-1].detach())

    history['lossD'] += [lossD.item()]
    history['lossG'] += [lossG.item()]
    history['avgStart'] += [avgStart.item()]
    history['avgEnd'] += [avgEnd.item()]
    history['valid'] += [valid.item()]
    history['validHist'] += [sum(history['valid'][-20:]) / len(history['valid'][-20:])]
    history['epoch'] += [epoch]
    history['porcentaje'] += [perc]
    


Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/cmougan/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/cmougan/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/cmougan/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/cmougan/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/Users/cmougan/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/cmougan/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/cmougan/anaconda3/lib/python3.6/multiprocessing/co

KeyboardInterrupt: 

In [ ]:
pen.size()

In [ ]:
pen = lmbda * (channel_penalty(aa) + duplicate_penalty(aa))
print(pen.size())
pen.unsqueeze_(-1)
pen.expand(128,3,10)
pen.size()


In [ ]:
aa.size()

In [ ]:
extra = Variable(torch.zeros(128, 1,18,11).cuda())

torch.cat((aa,extra), 1).size()

In [ ]:
extra = Variable(torch.zeros(128, 1,18,11).cuda())                     
# We add the penalty
pen = lmbda * (channel_penalty(aa) + duplicate_penalty(aa))
torch.cat((aa,extra), 1).size()

In [ ]:
extra.size()

In [ ]:
pen.unsqueeze(-1).unsqueeze(-1).size()

In [ ]:
pen.unsqueeze(-1).unsqueeze(-1).expand(128,1,18,11).size()


In [ ]:
pen.expand_as(extra)

In [ ]:
up=nn.UpsamplingNearest2d(size=(18,11))
up(pen)

In [ ]:
inp = Variable(torch.randn(10, 3, 24, 24))
m = nn.UpsamplingNearest2d(size=(55, 55))
out = m(inp)
out.size()